In [ ]:
import os
import time
import h5py
import math
import pickle
import numpy as np
import pandas as pd
import cv2
import threading
import queue
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import misc
from sklearn import model_selection, preprocessing, metrics
from sklearn.utils import shuffle
from skimage import transform
from tqdm import tqdm
from keras.regularizers import l2
from keras.models import Model, load_model
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import backend as K
from keras.losses import binary_crossentropy
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
from tensorflow.python.client import device_lib

%matplotlib inline

In [ ]:
DATA_PATH = '/kaggle/dev/carvana-image-masking-challenge-data'
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw_data')
TRAIN_PATH = os.path.join(RAW_DATA_PATH, 'train')
TEST_PATH = os.path.join(RAW_DATA_PATH, 'test')
TRAIN_MASKS_PATH = os.path.join(RAW_DATA_PATH, 'train_masks')
#TRAIN_MASKS_FIXED_PATH = os.path.join(DATA_PATH, 'fixed_masks/fix-HCK')
TRAIN_MASKS_CSV_PATH = os.path.join(RAW_DATA_PATH, 'train_masks.csv')
SAMPLE_SUBMISSION_PATH = os.path.join(RAW_DATA_PATH, 'sample_submission.csv')
METADATA_PATH = os.path.join(RAW_DATA_PATH, 'metadata.csv')
SUBMISSION_PATH = os.path.join(DATA_PATH, 'submissions')
ASSETS_PATH = os.path.join(DATA_PATH, 'assets')
MODELS_PATH = os.path.join(ASSETS_PATH, 'models')
TENSORBOARD_PATH = os.path.join(ASSETS_PATH, 'tensorboard')

In [ ]:
train_masks_df = pd.read_csv(TRAIN_MASKS_CSV_PATH)
print('train_masks_df.shape', train_masks_df.shape)
train_masks_df.head()

In [ ]:
# Constants
HEIGHT_ORIG = 1280
WIDTH_ORIG = 1918
CHANNELS_ORIG = 3

HEIGHT = 1024
WIDTH = 1024
CHANNELS = 3
new_shape = (HEIGHT, WIDTH, CHANNELS)
mask_shape = (new_shape[0], new_shape[1], 1)

def get_img_id(img_path):
    return img_path[:15]

img_ids = list(map(get_img_id, list(train_masks_df.img.values)))

def load_image_disk(img_id, folder=TRAIN_PATH):
    img = misc.imread(os.path.join(folder, img_id + ".jpg"))
    return img

def get_image(img_id):
    return train_imgs[img_id]

# Return mask as 1/0 binary img with single channel
def load_mask_disk(img_id, folder=TRAIN_MASKS_PATH, filetype='gif'):
    mask = misc.imread(os.path.join(folder,  "{}_mask.{}".format(img_id, filetype)), flatten=True)
    mask[mask > 128] = 1
    if len(mask.shape) == 2:
        mask = mask.reshape(mask.shape[0], mask.shape[1], 1)
    return mask

def get_mask(img_id):
    return train_masks[img_id]

# Helper functions to plot car, mask, masked_car
def plot_image(img_id):
    img = misc.imread(os.path.join(TRAIN_PATH, img_id + ".jpg"))
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.show()
    
def plot_mask(img_id, folder=TRAIN_MASKS_PATH, filetype='gif', ax=None):
    mask = misc.imread(os.path.join(folder, "{}_mask.{}".format(img_id, filetype)))
    if ax == None:
        imgplot = plt.imshow(mask)
        plt.axis('off')
        plt.show()
    else:
        ax.imshow(mask)
        ax.axis('off')
    
def plot_masked_image(img_id, ax=None):
    img = misc.imread(os.path.join(TRAIN_PATH, img_id + ".jpg"))
    mask = misc.imread(os.path.join(TRAIN_MASKS_PATH, img_id + "_mask.gif"))
    mask = mask[:,:,0:3]
    mask[mask == 255] = 1 
    masked_img = img * mask
    if ax == None:
        imgplot = plt.imshow(masked_img)
        plt.axis('off')
        plt.show()
    else:
        ax.imshow(masked_img)
        ax.axis('off')
    
def resize_img(img, new_s = new_shape):
    return transform.resize(img, new_s)

In [ ]:
# Read training images into memory
train_imgs = {}
for img_path in tqdm(os.listdir(TRAIN_PATH)):
    img_id = get_img_id(img_path)
    train_imgs[img_id] = cv2.resize(load_image_disk(img_id), (new_shape[0], new_shape[1]))

In [ ]:
# Read training masks into memory
train_masks = {}
for img_path in tqdm(os.listdir(TRAIN_MASKS_PATH)):
    img_id = get_img_id(img_path)
    train_masks[img_id] = np.expand_dims(cv2.resize(load_mask_disk(img_id), (new_shape[0], new_shape[1])), axis=2)

In [ ]:
# def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
#                              sat_shift_limit=(-255, 255),
#                              val_shift_limit=(-255, 255), u=0.5):
#     if np.random.random() < u:
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
#         h, s, v = cv2.split(image)
#         hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
#         h = cv2.add(h, hue_shift)
#         sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
#         s = cv2.add(s, sat_shift)
#         val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
#         v = cv2.add(v, val_shift)
#         image = cv2.merge((h, s, v))
#         image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
#     return image

# def randomShiftScaleRotate(image, mask,
#                            shift_limit=(-0.0625, 0.0625),
#                            scale_limit=(-0.1, 0.1),
#                            rotate_limit=(-45, 45), aspect_limit=(0, 0),
#                            borderMode=cv2.BORDER_REFLECT_101, u=0.5):
#     if np.random.random() < u:
#         height, width, channel = image.shape

#         angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
#         scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
#         aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
#         sx = scale * aspect / (aspect ** 0.5)
#         sy = scale / (aspect ** 0.5)
#         dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
#         dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

#         cc = np.math.cos(angle / 180 * np.math.pi) * sx
#         ss = np.math.sin(angle / 180 * np.math.pi) * sy
#         rotate_matrix = np.array([[cc, -ss], [ss, cc]])

#         box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
#         box1 = box0 - np.array([width / 2, height / 2])
#         box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

#         box0 = box0.astype(np.float32)
#         box1 = box1.astype(np.float32)
#         mat = cv2.getPerspectiveTransform(box0, box1)
#         image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
#                                     borderValue=(0, 0, 0,))
#         mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
#                                    borderValue=(0, 0, 0,))
#         if len(mask.shape) == 2:
#             mask = np.expand_dims(mask, axis=2)

#     return image, mask

# def randomHorizontalFlip(image, mask, u=0.5):
#     if np.random.random() < u:
#         image = cv2.flip(image, 1)
#         mask = cv2.flip(mask, 1)

#     return image, mask

In [ ]:
def generate_training_batch(data, batch_size):
    while True:
        X_batch = []
        Y_batch = []
        batch_ids = np.random.choice(data,
                                     size=batch_size,
                                     replace=False)
        for idx, img_id in enumerate(batch_ids):
            x = get_image(img_id)
            y = get_mask(img_id)
#             x, y = randomShiftScaleRotate(x, y,
#                                           shift_limit=(-0.0625, 0.0625),
#                                           scale_limit=(-0.1, 0.1),
#                                           rotate_limit=(-0, 0))
#             x = randomHueSaturationValue(x,
#                                hue_shift_limit=(-50, 50),
#                                sat_shift_limit=(-5, 5),
#                                val_shift_limit=(-15, 15))
            X_batch.append(x)
            Y_batch.append(y)
        X = np.asarray(X_batch, dtype=np.float32)
        Y = np.asarray(Y_batch, dtype=np.float32)
        yield X, Y

def generate_validation_batch(data, batch_size):
    while True:
        X_batch = []
        Y_batch = []
        batch_ids = np.random.choice(data,
                                     size=batch_size,
                                     replace=False)
        for idx, img_id in enumerate(batch_ids):
            x = get_image(img_id)
            y = get_mask(img_id)
            X_batch.append(x)
            Y_batch.append(y)
        X = np.asarray(X_batch, dtype=np.float32)
        Y = np.asarray(Y_batch, dtype=np.float32)
        yield X, Y

def generate_validation_data_seq(data):
    idx = 0
    while True:
        img_id = data[idx]
        X = get_image(img_id)
        Y = get_mask(img_id)
        yield img_id, X, Y
        idx  += 1
        if idx >= len(data):
            break

In [ ]:
# Visualize impact of random shift scale rotate
random_idx = np.random.randint(len(train_imgs.keys()))
random_img_id = list(train_imgs.keys())[random_idx]
plot_image(random_img_id)
temp_img = get_image(random_img_id)
mask = get_mask(random_img_id)
temp_img, temp_mask = randomShiftScaleRotate(temp_img, mask,
                              shift_limit=(-0.0625, 0.0625),
                              scale_limit=(-0.1, 0.1),
                              rotate_limit=(-0, 0))

plt.imshow(temp_img * gray2rgb(temp_mask))
plt.axis('off')
plt.show()

In [ ]:
# Visualize impact of random hue saturation
# random_idx = np.random.randint(len(train_imgs.keys()))
# random_img_id = list(train_imgs.keys())[random_idx]
plot_image(random_img_id)
temp_img = cv2.imread(os.path.join(TRAIN_PATH, '{}.jpg'.format(random_img_id)))
temp_img = randomHueSaturationValue(temp_img,
                               hue_shift_limit=(-50, 50),
                               sat_shift_limit=(-5, 5),
                               val_shift_limit=(-15, 15))
plt.imshow(temp_img)
plt.axis('off')
plt.show()

In [ ]:
def down(filters, input_):
    down_ = Conv2D(filters, (3, 3), padding='same')(input_)
    down_ = BatchNormalization()(down_)
    down_ = Activation('relu')(down_)
    down_ = Conv2D(filters, (3, 3), padding='same')(down_)
    down_ = BatchNormalization()(down_)
    down_res = Activation('relu')(down_)
    down_pool = MaxPooling2D((2, 2), strides=(2, 2))(down_)
    return down_pool, down_res

def up(filters, input_, down_):
    up_ = UpSampling2D((2, 2))(input_)
    up_ = concatenate([down_, up_], axis=3)
    up_ = Conv2D(filters, (3, 3), padding='same')(up_)
    up_ = BatchNormalization()(up_)
    up_ = Activation('relu')(up_)
    up_ = Conv2D(filters, (3, 3), padding='same')(up_)
    up_ = BatchNormalization()(up_)
    up_ = Activation('relu')(up_)
    up_ = Conv2D(filters, (3, 3), padding='same')(up_)
    up_ = BatchNormalization()(up_)
    up_ = Activation('relu')(up_)
    up_ = Conv2D(filters, (3, 3), padding='same')(up_)
    up_ = BatchNormalization()(up_)
    up_ = Activation('relu')(up_)
    return up_

In [ ]:
def get_unet_1024(input_shape=(HEIGHT, WIDTH, CHANNELS), num_classes=1):
    inputs = Input(shape=input_shape)

    down0b, down0b_res = down(8, inputs)
    down0a, down0a_res = down(16, down0b)
    down0, down0_res = down(32, down0a)
    down1, down1_res = down(64, down0)
    down2, down2_res = down(128, down1)
    down3, down3_res = down(256, down2)
    down4, down4_res = down(512, down3)
    
    center = Conv2D(1024, (3, 3), padding='same')(down4)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)
    center = Conv2D(1024, (3, 3), padding='same')(center)
    center = BatchNormalization()(center)
    center = Activation('relu')(center)

    up4 = up(512, center, down4_res)
    up3 = up(256, up4, down3_res)
    up2 = up(128, up3, down2_res)
    up1 = up(64, up2, down1_res)
    up0 = up(32, up1, down0_res)
    up0a = up(16, up0, down0a_res)
    up0b = up(8, up0a, down0b_res)

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid', name='final_layer')(up0b)

    model = Model(inputs=inputs, outputs=classify)

    return model

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# def dice_coef_v2(y_true, y_pred, smooth=1):
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     y_pred_f = K.greater_equal(y_pred_f, 0.5)
    
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

In [ ]:
# def weighted_bce_loss(y_true, y_pred, weight):
#     # avoiding overflow
#     epsilon = 1e-7
#     y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
#     logit_y_pred = K.log(y_pred / (1. - y_pred))
    
#     # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
#     loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
#     (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
#     return K.sum(loss) / K.sum(weight)

# def weighted_dice_coef(y_true, y_pred, weight):
#     smooth = 1.
#     w, m1, m2 = weight * weight, y_true, y_pred
#     intersection = (m1 * m2)
#     score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
#     return score

# def weighted_dice_loss(y_true, y_pred, weight):
#     return 1. - weighted_dice_coef(y_true, y_pred, weight)

# def weighted_bce_dice_loss(y_true, y_pred):
#     y_true = K.cast(y_true, 'float32')
#     y_pred = K.cast(y_pred, 'float32')
#     # if we want to get same size of output, kernel size must be odd number
#     averaged_mask = K.pool2d(
#             y_true, pool_size=(41, 41), strides=(1, 1), padding='same', pool_mode='avg')
#     border = K.cast(K.greater(averaged_mask, 0.01), 'float32') * K.cast(K.less(averaged_mask, 0.99), 'float32')
#     weight = K.ones_like(averaged_mask)
#     w0 = K.sum(weight)
#     weight += border * 2
#     w1 = K.sum(weight)
#     weight *= (w0 / w1)
#     loss = weighted_bce_loss(y_true, y_pred, weight) + \
#     weighted_dice_loss(y_true, y_pred, weight)
#     return loss

In [ ]:
BATCH_SIZE = 4

In [ ]:
ts = str(int(time.time()))
model_name = 'malhot'
num_epochs = 30
steps_per_epoch = 1018
run_name = 'model={}-batch_size={}-num_epoch={}-steps_per_epoch={}-ts={}'.format(model_name,
                                                                          BATCH_SIZE,
                                                                          num_epochs,
                                                                          steps_per_epoch,
                                                                          ts)
tensorboard_loc = os.path.join(TENSORBOARD_PATH, run_name)
checkpoint_loc = os.path.join(MODELS_PATH, 'model-{}-weights.h5'.format(ts))

earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience=2, 
                              verbose=1, 
                              min_delta = 0.0001,
                              mode='min',)

modelCheckpoint = ModelCheckpoint(checkpoint_loc,
                                  monitor = 'val_loss', 
                                  save_best_only = True, 
                                  mode = 'min', 
                                  verbose = 1,
                                  save_weights_only = True)

tensorboard = TensorBoard(log_dir=tensorboard_loc, histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [modelCheckpoint, earlyStopping, tensorboard]

model = get_unet_1024()
model.compile(loss=bce_dice_loss, optimizer=Adam(lr=0.0001), metrics=[dice_coef])
model.summary()

In [ ]:
train_ids, validation_ids = model_selection.train_test_split(img_ids, random_state=42, test_size=0.20)
train_generator = generate_training_batch(train_ids, BATCH_SIZE)
valid_generator = generate_validation_batch(validation_ids, BATCH_SIZE)
VALIDATION_STEPS = int(len(validation_ids) / BATCH_SIZE)

In [ ]:
# model = get_unet_1024()
# model.load_weights(os.path.join(MODELS_PATH, 'model-1505273667-weights.h5'))
# model.compile(loss=bce_dice_loss, optimizer=Adam(1e-5), metrics=[dice_coef])

In [ ]:
print('Starting run {}'.format(run_name))
history = model.fit_generator(
        train_generator, 
        steps_per_epoch = steps_per_epoch, 
        epochs = num_epochs,
        callbacks = callbacks_list,
        verbose = 1,
        validation_data = valid_generator,
        validation_steps = VALIDATION_STEPS)

model_path = os.path.join(MODELS_PATH, 'model-{}.h5'.format(ts))
history_path = os.path.join(MODELS_PATH, 'model-{}.history'.format(ts))
model.save(model_path)
pickle.dump(history.history, open(history_path, "wb"))
print('Saved model at {}'.format(model_path))
print('Saved model history at {}'.format(history_path))

In [ ]:
model = load_model(os.path.join(MODELS_PATH, 'model-1505333320.h5'), custom_objects={'bce_dice_loss': bce_dice_loss,
                                                                                    'dice_coef': dice_coef})

In [ ]:
model.evaluate_generator(valid_generator, VALIDATION_STEPS)

### Error analysis on validation set

In [ ]:
validation_dices = []
for img_id, X, Y in generate_validation_data_seq(validation_ids):
    error = model.evaluate(np.expand_dims(X, axis=0), np.expand_dims(Y, axis=0), verbose=0)
    validation_dices.append((img_id, error[0], error[1]))

val_eval_df = pd.DataFrame.from_records(validation_dices, columns=['img_id', 'val_loss', 'dice_coef'])
val_eval_df.head()

In [ ]:
val_eval_df.to_csv(os.path.join(ASSETS_PATH, 'val_eval_df-1505408279.csv'), index=False)

In [ ]:
val_eval_df = pd.read_csv(os.path.join(ASSETS_PATH, 'val_eval_df-1505408279.csv'))

In [ ]:
plt.figure(figsize=(20,10))
sns.distplot(val_eval_df['dice_coef'], kde=False)

In [ ]:
np.median(val_eval_df['dice_coef'])

In [ ]:
np.mean(val_eval_df['dice_coef'])

In [ ]:
np.std(val_eval_df['dice_coef'])

In [ ]:
def get_outliers(data, col, m=2):
    return data[(data[col] - np.mean(data[col])) < -1.0 * (m * np.std(data[col]))]

In [ ]:
outlier_df = get_outliers(val_eval_df, 'dice_coef', 2)
outlier_df = outlier_df.sort_values('dice_coef')
print(outlier_df.shape)
outlier_df

In [ ]:
for img_id in outlier_df.img_id.tolist():
    print(img_id, outlier_df[outlier_df.img_id == img_id].values.tolist()[0][2])
    test_img = get_image(img_id)

    # Plot original image
    actual_img = load_image_disk(img_id)
    plt.figure(figsize=(15, 15))
    plt.imshow(actual_img)
    plt.axis('off')
    
    # Predict mask
    pred_mask = model.predict(np.expand_dims(test_img, axis=0))
    
    # Plot predicted mask
    pred_mask = np.squeeze(pred_mask)    
    pred_mask = resize_img(pred_mask, (HEIGHT_ORIG, WIDTH_ORIG))
    pred_mask[pred_mask <= 0.5] = 0
    pred_mask[pred_mask > 0.5] = 1
    
    # Plot ground truth mask
    mask = load_mask_disk(img_id)
    mask = np.squeeze(mask)
    mask = resize_img(mask, (HEIGHT_ORIG, WIDTH_ORIG))
        
    # Plot intersection (true_positives) of two masks
    true_positives = pred_mask * mask
    
    # Plot false positives (pred_mask == 1 && mask == 0)
    false_positives = np.zeros_like(mask)
    false_positives[(pred_mask == 1) & (mask == 0)] = 1
    
    # Plot false negatives (pred_mask == 0 && mask == 1)
    false_negatives = np.zeros_like(mask)
    false_negatives[(pred_mask == 0) & (mask == 1)] = 1

    # Plot true negatives (pred_mask == 0 && mask == 0)
    true_negatives = np.zeros_like(mask)
    true_negatives[(pred_mask == 0) & (mask == 0)] = 1
    
    # Plot merged mask 
    # Legend: 
    #   Red: false positives 
    #   Green: true positives
    #   Blue: false negatives
    #   Black: true negatives
    #   White: background (unclassified pixels) - this should never be visible
    rgb_merged_mask = np.zeros((HEIGHT_ORIG, WIDTH_ORIG, CHANNELS_ORIG))
    rgb_merged_mask = 255 # White
    rgb_true_positives = gray2rgb(true_positives)
    rgb_false_positives = gray2rgb(false_positives)
    rgb_false_negatives = gray2rgb(false_negatives)
    rgb_true_negatives = gray2rgb(true_negatives)

    rgb_merged_mask = rgb_true_positives + rgb_false_positives + rgb_false_negatives + rgb_true_negatives
    rgb_merged_mask[true_positives == 1] = [0, 255, 0] # Green
    rgb_merged_mask[false_positives == 1] = [255, 0, 0] # Red
    rgb_merged_mask[false_negatives == 1] = [0, 0, 255] # Blue
    rgb_merged_mask[true_negatives == 1] = [0, 0, 0] # Black
    
    plt.figure(figsize=(15, 15))
    plt.imshow(rgb_merged_mask)
    plt.axis('off')
    plt.show()

In [ ]:
# Helper methods
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of 
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask, 
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def gray2rgb(img):
    img = np.squeeze(img)
    w, h = img.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = img
    ret[:, :, 1] = img
    ret[:, :, 2] = img
    return ret

def load_imgs(img_ids, folder=TRAIN_PATH):
    imgs = []
    for img_id in img_ids:
        img = misc.imread(os.path.join(folder, img_id + ".jpg"))
        imgs.append(img)
    return np.asarray(imgs)

def resize_imgs(imgs, factor=0.1):
    resized_imgs = []
    for img in imgs:
        resized_img = rescale(img, factor)
        resized_imgs.append(resized_img)
    return np.asarray(resized_imgs)

def rescale_and_clean_masks(masks):
    clean_masks = np.zeros((masks.shape[0], HEIGHT_ORIG, WIDTH_ORIG, 1), dtype=np.uint8)
    for i in range(masks.shape[0]):        
        mask = resize(masks[i], (HEIGHT_ORIG, WIDTH_ORIG, 1))
        mask[mask <= 0.5] = 0
        mask[mask > 0.5] = 1
        clean_masks[i] = mask
    return clean_masks

def rle_masks(masks):  
    rles = []
    for i in range(masks.shape[0]):
        rles.append(rle_to_string(rle_encode(masks[i])))
    return rles

In [ ]:
model = load_model(os.path.join(MODELS_PATH, 'model-1505333320.h5'), custom_objects={'bce_dice_loss': bce_dice_loss,
                                                                                    'dice_coef': dice_coef})

In [ ]:
model = get_unet_1024()
model.load_weights(os.path.join(MODELS_PATH, 'model-1505408279-weights.h5'))
model.compile(loss=bce_dice_loss, optimizer=Adam(1e-5), metrics=[dice_coef])

In [ ]:
for img_path in os.listdir(TEST_PATH)[1000:1005]:
    #print(img_id, outlier_df[outlier_df.img_id == img_id].values.tolist()[0][2])
    #test_img = get_image(img_id)
    img_id = get_img_id(img_path)
    
    # Plot original image
    actual_img = load_image_disk(img_id, TEST_PATH)
    plt.figure(figsize=(15, 15))
    plt.imshow(actual_img)
    plt.axis('off')
    
    test_img_cv2 = cv2.resize(actual_img, (new_shape[0], new_shape[1]))
    
    plt.figure(figsize=(15, 15))
    plt.imshow(test_img_r)
    plt.axis('off')
    
    plt.figure(figsize=(15, 15))
    plt.imshow(test_img_cv2)
    plt.axis('off')
    
    # Predict mask
    pred_mask = model.predict(np.expand_dims(test_img_cv2, axis=0))
    
    # Plot predicted mask
    pred_mask = np.squeeze(pred_mask)    
    pred_mask = resize_img(pred_mask, (HEIGHT_ORIG, WIDTH_ORIG))
    pred_mask[pred_mask <= 0.5] = 0
    pred_mask[pred_mask > 0.5] = 1
    
    plt.figure(figsize=(15, 15))
    plt.imshow(pred_mask)
    plt.axis('off')
    plt.show()

In [ ]:
# TEST_BATCH_SIZE = 16
# NUM_BATCHES = int(math.ceil(len(test_img_ids) / TEST_BATCH_SIZE))

# result = []
# for batch in tqdm(range(NUM_BATCHES)):
#     start = batch * TEST_BATCH_SIZE
#     end = start + TEST_BATCH_SIZE
#     end = len(test_img_ids) if end > len(test_img_ids) else end
#     test_imgs = load_imgs(test_img_ids[start:end], TEST_PATH)
#     resized_test_imgs = resize_imgs(test_imgs)
#     pred_masks = model.predict(resized_test_imgs)
#     clean_masks = rescale_and_clean_masks(pred_masks)
#     rles = rle_masks(clean_masks)
#     result.extend(zip(test_img_paths[start:end], rles))

# sub_ts = str(int(time.time()))
# submission_df = pd.DataFrame.from_records(result, columns=['img', 'rle_mask'])
# sub_file_path = os.path.join(SUBMISSION_PATH, 'malhot-submission-{}.csv'.format(sub_ts))
# submission_df.to_csv(sub_file_path, index=False)
# print('Generated submission {}'.format(sub_file_path))

In [ ]:
# Reduce random dots in output masks
# new_result = []
# for idx in tqdm(range(len(result))):
#     img_path = result[idx][0]
#     mask = result[idx][1]
#     rle_decoded_mask = rle_decode(mask, (HEIGHT_ORIG, WIDTH_ORIG, 1))
#     limg = measure.label(rle_decoded_mask)
#     props = measure.regionprops(limg)
#     props = sorted(props, key=lambda p: -p.area)
#     # Erase all except the biggest blob
#     rle_decoded_mask &= (limg == props[0].label)
#     new_result.append((img_path, rle_to_string(rle_encode(rle_decoded_mask))))

In [ ]:
# sub_ts = str(int(time.time()))
# submission_df = pd.DataFrame.from_records(new_result, columns=['img', 'rle_mask'])
# sub_file_path = os.path.join(SUBMISSION_PATH, 'malhot-submission-{}.csv'.format(sub_ts))
# submission_df.to_csv(sub_file_path, index=False)
# print('Generated submission {}'.format(sub_file_path))

In [ ]:
# Multi GPU inference code

sample_submission_df = pd.read_csv(SAMPLE_SUBMISSION_PATH)
input_size = HEIGHT
batch_size = 4
orig_width = WIDTH_ORIG
orig_height = HEIGHT_ORIG
threshold = 0.5

gpus = [x.name for x in device_lib.list_local_devices() if x.name[:4] == '/gpu']

df_test = pd.read_csv(SAMPLE_SUBMISSION_PATH)
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

names = []
rles = []
q_size = 10

for id in ids_test:
    names.append('{}.jpg'.format(id))

# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle

def create_model(gpu):
    with tf.device(gpu):
        model = get_unet_1024()
    model.load_weights(os.path.join(MODELS_PATH, 'model-1505333320-weights.h5'))
    return model

def data_loader(q, ):
    for start in tqdm(range(0, len(ids_test), batch_size)):
        x_batch = []
        end = min(start + batch_size, len(ids_test))
        ids_test_batch = ids_test[start:end]
        for id in ids_test_batch.values:
            img = load_image_disk(id, TEST_PATH)
            if input_size is not None:
                img = cv2.resize(img, (input_size, input_size))
            x_batch.append(img)
        x_batch = np.array(x_batch, np.float32)
        q.put((ids_test_batch, x_batch))
    for g in gpus:
        q.put((None, None))

def predictor(q, gpu):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    with sess.as_default():
        model = create_model(gpu)
        while True:
            ids, x_batch = q.get()
            if ids is None:
                break
            preds = model.predict_on_batch(x_batch)
            preds = np.squeeze(preds, axis=3)
            for i,pred in enumerate(preds):
                if input_size is not None:
                    prob = cv2.resize(pred, (orig_width, orig_height))
                else:
                    prob = pred
                mask = prob > threshold
                rle = run_length_encode(mask)
                id = ids.iloc[i]
                rles.append((id, rle))

print('Predicting on {} samples with batch_size = {}...'.format(len(ids_test), batch_size))
q = queue.Queue(maxsize=q_size)
threads = []
threads.append(threading.Thread(target=data_loader, name='DataLoader', args=(q,)))
threads[0].start()
for gpu in gpus:
    print("Starting predictor at device " + gpu)

    t = threading.Thread(target=predictor, name='Predictor', args=(q, gpu))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

sub_ts = str(int(time.time()))
submission_df = pd.DataFrame(rles, columns=['img', 'rle_mask'])
submission_df['img'] += '.jpg'
sub_file_path = os.path.join(SUBMISSION_PATH, 'malhot-submission-{}.csv.gz'.format(sub_ts))
submission_df.to_csv(sub_file_path, index=False, compression='gzip')
print('Generated submission {}'.format(sub_file_path))